<a href="https://colab.research.google.com/github/loki20051267/Lokesh1267/blob/main/AIML_A3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from collections import defaultdict
import random

class Board(defaultdict):
    """A board has the player to move, a cached utility value, and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    empty = '.'
    off = '#'

    def __init__(self, width=3, height=3, to_move=None, **kwds):
        super().__init__(lambda: self.empty)  # Use a lambda function to provide the default value
        self.width = width
        self.height = height
        self.to_move = to_move
        self.utility = 0
        self.update(kwds)

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def missing(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

class TicTacToe:
    """Play TicTacToe on an ‘height’ by ‘width’ board, needing ‘k’ in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k  # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(width=width, height=height, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        available_actions = self.squares - set(board)
        print(f"Available actions: {available_actions}")  # Debug print
        return available_actions

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        new_board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = self.k_in_row(new_board, player, square)
        new_board.utility = (0 if not win else +1 if player == 'X' else -1)
        return new_board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        no_empty_squares = len(self.squares - set(board)) == 0
        return board.utility != 0 or no_empty_squares

    def display(self, board):
        print(board)

    def k_in_row(self, board, player, square):
        """True if player has k pieces in a line through square."""
        def in_row(x, y, dx, dy):
            return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)

        return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= self.k
                   for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

def random_player(game, state):
    """Choose a random action from the list of legal actions."""
    actions = list(game.actions(state))
    if not actions:
        raise ValueError("No actions available")
    return random.choice(actions)

def play_game(game):
    """Play a game using random players."""
    state = game.initial
    move_count = 0
    max_moves = game.initial.width * game.initial.height

    while not game.is_terminal(state) and move_count < max_moves:
        action = random_player(game, state)
        print(f"Player {state.to_move} moves at {action}")
        state = game.result(state, action)
        game.display(state)
        print()
        move_count += 1

    if game.is_terminal(state):
        result = 'Draw' if state.utility == 0 else f"{state.to_move} wins!"
    else:
        result = "Game did not end properly."

    print(f"Game Over. {result}")

# Play a game of Tic-Tac-Toe
game = TicTacToe()
play_game(game)


Available actions: {(0, 1), (1, 2), (2, 1), (0, 0), (1, 1), (2, 0), (0, 2), (2, 2), (1, 0)}
Player X moves at (1, 2)
. . .
. . .
. X .


Game Over. Draw
